In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import scipy

In [116]:
points = []
centroids = []
num_points = 100
num_centroids = 8
e=0.01

for i in range(num_points):
    points.append(np.random.normal())
for i in range(num_centroids):
    centroids.append(np.random.normal())

points = np.array(points)
centroids = np.array(centroids)

In [117]:
def grouping(point_array, centroid_array):
    #grouping the centroids into groups
    grouping = []
    return_list = {}
    centroid_array = np.sort(centroid_array)
    for i in range(len(centroid_array)-1):
        midpoint = (centroid_array[i]+centroid_array[i+1])/2
        grouping.append(midpoint)
    for i in range(len(grouping)+1):
        if i ==0:
            return_list[i] = point_array[np.where(point_array<grouping[i])]
        if i == len(grouping):
            return_list[i] = point_array[np.where(point_array>grouping[i-1])]
        if i!=0 and i!= len(grouping):
            return_list[i] = point_array[np.where(np.logical_and(point_array>grouping[i-1], point_array<grouping[i]))]
        
    return return_list

In [118]:
def new_centroids(points):
    new_centroid_points = []
    for i in range(len(points)):
        new_centroid_points.append(points[i].mean())
    return new_centroid_points

In [119]:
def distortion(points, centroids):
    total_dis = 0
    for i in range(len(points)):
        dis = np.square(np.array(points[i])-centroids[i]).sum() #distortion for each cluster
        total_dis = total_dis + dis
    return total_dis

In [120]:
def lloydsalgorithm(point_array, centroid_array):
    previous_centroids = centroid_array
    groups = grouping(point_array, centroid_array)
    
    new_centroid_array = new_centroids(groups)
    new_groups = grouping(point_array, new_centroid_array)

    prev_dis = distortion(groups, previous_centroids)
    new_dis = distortion(new_groups, new_centroid_array)

    if np.abs(prev_dis-new_dis)/prev_dis<e:
        return new_centroid_array
    else:
        return lloydsalgorithm(point_array, new_centroid_array)
    


In [121]:
lloydsalgorithm(points, centroids)

[-1.849441262603312,
 -1.1226437840944747,
 -0.5887316291831748,
 -0.11315092998736404,
 0.32219078111438626,
 0.6235222164027524,
 1.0326444193050897,
 2.045474634019054]